In [160]:
!pip install starter
import numpy as np
from random import getrandbits
from unittest import result
import tensorflow as tf
import matplotlib.pyplot as plt
from starter import loadData

trainData, validData, testData, trainTarget, validTarget, testTarget = loadData()
trainData = trainData.reshape(3500, -1)
validData = validData.reshape(100, -1)
testData = testData.reshape(145, -1)
trainData = tf.convert_to_tensor(trainData, dtype=tf.float32)
validData = tf.convert_to_tensor(validData, dtype=tf.float32)
trainTarget = tf.convert_to_tensor(trainTarget, dtype=tf.float32)
validTarget = tf.convert_to_tensor(validTarget, dtype=tf.float32)
testData = tf.convert_to_tensor(testData, dtype=tf.float32)
testTarget = tf.convert_to_tensor(testTarget, dtype=tf.float32)
N, D = trainData.shape
trainLoss = []
validLoss = []
trainAcc = []
validAcc = []

In [155]:
def ce_loss(y, yHat):
  yHat = tf.clip_by_value(yHat, 1e-9, 1.)
  return tf.reduce_mean(-tf.reduce_sum(- y * tf.math.log(yHat) - (1 - y) * tf.math.log(1 - yHat)))

# sigmoid function
def sigmoid(z):
  return 1 / (1 + np.exp(-z))

# predict y value and returns the probability
# the return value is a [n * d][d * 1] = [n * 1] array
def predict(w, b, x):
  return sigmoid(x @ w + b)

# plot
def plot_loss(alpha):

  plt.figure()
  plt.plot(trainLoss, label='Training Loss')
  plt.plot(validLoss, label='Validation Loss')
  title = "Traing & Validation Loss vs epochs @ " + "%f" %alpha
  plt.title(title)
  plt.xlabel("epoches")
  plt.ylabel("loss")
  plt.legend()
  plt.draw()

  return

def plot_accuracy(reg):

  plt.figure()
  plt.plot(trainAcc, label='Training Accuracy')
  plt.plot(validAcc, label='Validation Accuracy')
  title = "Traing & Validation Accuracy vs epochs @ " + "%f" %reg
  plt.title(title)
  plt.xlabel("epoches")
  plt.ylabel("accuracy")
  plt.legend()
  plt.draw()

  return 

'''
def accuracy(y, yHat):
  correctPrediction = tf.equal(tf.argmax(yHat, 1), tf.argmax(y, 1))
  accuracy = tf.reduce_mean(tf.cast(correctPrediction, tf.float32))

  return accuracy
'''
def accuracy(y, yHat):
  targetHat = (yHat.numpy() > 0.5).astype(float)
  acc = (targetHat == y.numpy()).mean()
  y = tf.convert_to_tensor(y, dtype=tf.float32)
  yHat = tf.convert_to_tensor(yHat, dtype=tf.float32)

  return acc



In [156]:
def shuffle_batch(batchSize):
  slicedDb = tf.data.Dataset.from_tensor_slices((trainData, trainTarget))
  slicedDb = slicedDb.shuffle(5000).batch(batchSize)

  return slicedDb

def init_w_b():
  w = tf.Variable(tf.random.truncated_normal(shape=(D, 1)), name="weight")
  b = tf.Variable(tf.zeros(shape=1), name="bias")

  return w, b

def train_batch(w, b, x, y, reg, optimizer):
  with tf.GradientTape(persistent=True) as g:
    yHat = tf.nn.sigmoid(tf.add(tf.matmul(x, w), b))
    loss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits = yHat, labels = y) + (reg / 2) * tf.norm(w) ** 2)
  
  grad = g.gradient(loss, [w, b])
  optimizer.apply_gradients(zip(grad, [w, b]))

  return w, b

In [ ]:
# main
batchSizeList = [100, 500, 700, 1750]
w, b = init_w_b()
bt1 = [0.95, 0.99]
bt2 = [0.99, 0.9999]
e = [1e-09, 1e-04]
optimizer = tf.compat.v1.train.AdamOptimizer(learning_rate=0.001, beta2=bt2[1])

trainLoss.clear()
validLoss.clear()
trainAcc.clear()
validLoss.clear()

for i in range(700):
  db = shuffle_batch(batchSizeList[1])
  for step, (batchX, batchY) in enumerate(db):
    w, b = train_batch(w, b, batchX, batchY, 0, optimizer)
  
  yHat = tf.nn.sigmoid(tf.add(tf.matmul(trainData, w), b))
  loss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits = yHat, labels = trainTarget) + (0 / 2) * tf.norm(w) ** 2)
  trainLoss.append(loss)
  trainAcc.append(accuracy(trainTarget, yHat))

  yHatValid = tf.nn.sigmoid(tf.add(tf.matmul(validData, w), b))
  lossValid = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits = yHatValid, labels = validTarget) + (0 / 2) * tf.norm(w) ** 2)
  validLoss.append(lossValid)
  validAcc.append(accuracy(validTarget, yHatValid))

plot_loss(0)
plot_accuracy(0)

yHatTest = tf.nn.sigmoid(tf.add(tf.matmul(testData, w), b))
lossTest = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits = yHatTest, labels = testTarget) + (0 / 2) * tf.norm(w) ** 2)
testAcc = accuracy(testTarget, yHatTest)

print(trainAcc[-1])
print(" ")
print(validAcc[-1])
print(" ")
print(testAcc)
